<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/test0525.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt
!pip install optuna > log.txt

Cloning into 'c620'...
remote: Enumerating objects: 2606, done.
remote: Counting objects: 100% (336/336), done.
remote: Compressing objects: 100% (305/305), done.
remote: Total 2606 (delta 212), reused 60 (delta 30), pack-reused 2270
Receiving objects: 100% (2606/2606), 66.95 MiB | 30.54 MiB/s, done.
Resolving deltas: 100% (1705/1705), done.


In [2]:
from c620.FV2 import AllSystem
from c620.configV2 import config
import joblib
import numpy as np
import pandas as pd
import os
os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
Already up to date.


In [3]:
icg_path = '/content/drive/MyDrive/台塑輕油案子/data/c620/工廠測試資料_模組INPUT_0525/icg_input_0525.csv'
c620_path = '/content/drive/MyDrive/台塑輕油案子/data/c620/工廠測試資料_模組INPUT_0525/c620_feed_0525.csv'
t651_path = '/content/drive/MyDrive/台塑輕油案子/data/c620/工廠測試資料_模組INPUT_0525/t651_feed_0525.csv'

In [4]:
import pandas as pd

icg_input = pd.read_csv(icg_path,index_col=0)
icg_input.index = ['0525']

c620_feed = pd.read_csv(c620_path,index_col=0)
c620_feed.index = ['0525']

t651_feed = pd.read_csv(t651_path,index_col=0)
t651_feed.index = ['0525']

# 原本SPEC

In [5]:
icg_input[['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr']]

,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr
0525,0.0


In [6]:
icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
0525,855.10575


In [7]:
icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
0525,78.489998


In [8]:
icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
0525,40.390682


# 試算

In [9]:
f = AllSystem(config)
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1,c620_side_體積流量 = f.inference(icg_input.copy(),c620_feed.copy(),t651_feed.copy())

In [10]:
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
c660_wt1.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000

681.9707338399779

# 推薦

In [11]:
dist_rate ,SideDraw_in_BZ ,nainbz ,c620_op2 ,c660_op2 ,c670_op2 ,c620_op_Δ ,c660_op_Δ ,c670_op_Δ = f.recommend(icg_input.copy(),c620_feed.copy(),t651_feed.copy())

dist_rate:0.0 nainbz:678.6843259186905


In [12]:
dist_rate

0.0

In [13]:
SideDraw_in_BZ

70

In [14]:
nainbz

678.6843259186905

In [15]:
c620_op2

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate Production Rate and Composition,Density_Sidedraw Production Rate and Composition,Density_Bottoms Production Rate and Composition,Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
0525,0.864295,0.426528,0.827697,0.878698,0.872164,97.02076,15.32741,14.578189,181.068253,195.175781


In [16]:
c660_op2

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate (Benzene Drag) Production Rate and Composition,Density_Sidedraw (Benzene )Production Rate and Composition,Density_Bottoms Production Rate and Composition,Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
0525,0.8766,0.591327,0.833525,0.883686,0.869884,177.408447,15.72832,16.6682,87.019394,90.607285


In [17]:
c670_op2

,Density_Distillate Production Rate and Composition,Density_Bottoms Production Rate and Composition,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
0525,0.870889,0.872133,315.732147,37.247761,37.186985,181.394714,228.510696


In [18]:
c620_op_Δ

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate Production Rate and Composition,Density_Sidedraw Production Rate and Composition,Density_Bottoms Production Rate and Composition,Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
0525,-0.00003,0.0,0.0,-0.000004,0.00001,0.000404,-0.014752,-0.19399,0.024475,-0.015762


In [19]:
c660_op_Δ

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate (Benzene Drag) Production Rate and Composition,Density_Sidedraw (Benzene )Production Rate and Composition,Density_Bottoms Production Rate and Composition,Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
0525,-0.00009,0.008405,-0.006057,-0.000024,4.768372e-07,26.97522,2.133386,2.004081,0.195213,-0.401123


In [20]:
c670_op_Δ

,Density_Distillate Production Rate and Composition,Density_Bottoms Production Rate and Composition,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
0525,9.536743e-07,0.000027,-1.021393,-0.045925,-0.019196,-0.096863,-0.031479
